<a href="https://colab.research.google.com/github/lephuocdat2000/Advanced-CV/blob/main/Dog-and-Cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from PIL import Image
import numpy as np
import zipfile
from io import StringIO
import imghdr
import cv2 
from sklearn.model_selection import train_test_split
from torchvision import models
import torch
from torchvision import transforms
import tensorflow as tf

In [3]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Resize(227),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [4]:
alexnet = models.alexnet(pretrained=True)
alexnet.classifier=alexnet.classifier[0:5]

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth


#Đọc file data 

In [52]:
path_to_zip_file = '/content/drive/MyDrive/Thị giác máy tính/Thị giác máy tính nâng cao/Dog and Cat.zip'
imgzip = zipfile.ZipFile(path_to_zip_file)
inflist = imgzip.infolist()
images=[]
labels=[]
for f in inflist:
     if '.jpg' in f.filename:
       ifile = imgzip.open(f)
       img = Image.open(ifile)
       #Cho vào mạng alexnet
       img_t=transform(img)
       batch_t = torch.unsqueeze(img_t,0)
       out = alexnet(batch_t)
       out=out.detach().numpy()
      #  image_as_array = np.array(img, np.uint8)
      #  image_as_array = cv2.resize(image_as_array,(227,227))
       images.append(out)
       if 'dog' in f.filename: labels.append(1)
       elif 'cat' in f.filename: labels.append(0)

In [53]:
images=np.array(images)
labels=np.array(labels)

Load pre-train model

In [58]:
X_train, X_test, y_train, y_test = train_test_split(images,labels, test_size=0.2, random_state=42)

In [62]:
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1]*X_train.shape[2])
X_test=X_test.reshape(X_test.shape[0],X_test.shape[1]*X_test.shape[2])

In [64]:
from sklearn.svm import SVC
model = SVC(gamma='auto')
model.fit(X_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [65]:
model.predict(X_test)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])